# Truecasing

In [ ]:
import os

def truecase(experiment_folder):
    script = "/home/samuel/Software/mosesdecoder/scripts/recaser/train-truecaser.perl"
    dev_set = "/home/samuel/Documents/research_and_development/tweet-norm-dev_"
    
    print("Learning the truecaser model...")
    for extension in ["source", "target"]:
        !{script} --corpus dev_set{extension}.txt --corpus {experiment_folder}paraphrases.clean.{extension}

# Language Model

In [80]:
def create_lm_counts(input_dir, output_dir):
    ''' Create language models counts for all the files in a directory '''
    
    filenames = os.listdir(input_dir)
    for filename in filenames:
        !/local/kurs/mt/srilm/bin/i686-m64/ngram-count -kndiscount -text {input_dir}{filename} -lm {ouput_dir}/{filename}.counts -order 5 
        #!head n1 {input_dir}{filename}
        print("Created LM counts for file {filename}".format(**locals()))


# Translation Model

In [45]:
def train_smt(experiment_folder):
    script = "/home/samuel/Software/mosesdecoder/scripts/training/train-model.perl"
    giza = "/home/samuel/Software/mosesdecoder/tools"
    lm_file = "/home/samuel/Documents/research_and_development/language_model/lm.gz"
    corpus = "{experiment_folder}paraphrases.clean".format(**locals())
    
    print("Training SMT model...")
    !{script} --corpus {corpus} --f source --e target --root-dir {experiment_folder}translation_model/ --lm 0:5:{lm_file} --external-bin-dir {giza} >logfile 2>&1
    print("SMT model trained.")
    
    
def tune_smt(experiment_folder):
    script = "/home/samuel/Software/mosesdecoder/scripts/training/mert-moses.pl"
    dev_source = "/home/samuel/Documents/research_and_development/dev_set/tweet-norm-dev_source.txt"
    dev_target = "/home/samuel/Documents/research_and_development/dev_set/tweet-norm-dev_target.txt"
    ini_file = "{experiment_folder}translation_model/model/moses.ini".format(**locals())
    tuning_dir = "{experiment_folder}tuning/".format(**locals())
    moses = "/home/samuel/Software/mosesdecoder/bin/moses"
    !mkdir -p {tuning_dir}
    
    print("Tuning SMT model...")
    !{script} {dev_source} {dev_target} {moses} {ini_file} --working-dir {tuning_dir} --mertdir /home/samuel/Software/mosesdecoder/bin >{tuning_dir}logfile.mert 2>&1 --decoder-flags="-threads 4"
    print("SMT model tuned.")

# Significance Test

In [51]:
def significance_test_pruning(experiment_folder):
    file = "{experiment_folder}paraphrases.clean."
    index_script = "/home/samuel/Software/salm/Bin/Linux/Index/IndexSA.O64"
    prune_script = "/home/samuel/Software/mosesdecoder/contrib/sigtest-filter/filter-pt"
    phrase_table_dir = "{experiment_folder}/tuning/filtered/"
    
    print("Getting indices...")
    for extension in ["source", "target"]:
        !{index_script} {file}{extension}
    
    print("Unziping phrasetable...")
    !zcat {phrase_table_dir}phrase-table.0-0.1.1.gz > phrase-table
    
    print("Prunning phrasetable...")
    !cat {phrase_table_dir}phrase-table | {prune_script} -e {file}target -f {file}source -l a+e -n 30 > {phrase_table_dir}phrase-table.pruned
    print("Prunned Phrasetable.")

# Testing Tuned Translation Model

In [52]:
def test_tuned(experiment_folder):
    !/home/samuel/Software/mosesdecoder/bin/moses -f /home/samuel/Documents/research_and_development/experiment4/tuning/moses.ini < /home/samuel/Documents/research_and_development/dev_set/tweet-norm-dev_source.txt > results


# Experiments

In [54]:
experiment_folder = "/home/samuel/Documents/research_and_development/experiment4/"
#train_smt(experiment_folder)
tune_smt(experiment_folder)
#significance_test_pruning(experiment_folder)

Tuning SMT model...
SMT model tuned.
